In [ ]:
!pip install konlpy #한글 형태소 분석 라이브러리

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/[DS13&14] 문제해결 빅데이터 PJT_2조/튜닝 완료 모델/소프트 보팅

/content/drive/.shortcut-targets-by-id/1l_AUpAUvG2xbujGqvp7y2aivnC4LEEtP/[DS13&14] 문제해결 빅데이터 PJT_2조/튜닝 완료 모델/소프트 보팅


In [ ]:
import pickle
from tensorflow.keras.models import load_model
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from gensim.models import FastText
okt = Okt()

In [ ]:
FIRST_HANGUL = 0xAC00 #'가'
LAST_HANGUL = 0xD7A3 #'힣'
NO_JONGSUNG = 'ᴕ'

CHOSUNGS = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
JOONGSUNGS = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
JONGSUNGS = [NO_JONGSUNG,  'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

N_CHOSUNGS = 19
N_JOONGSUNGS = 21
N_JONGSUNGS = 28

def to_jamo(s):        
    result = []
    for c in s:
        if ord(c) < FIRST_HANGUL or ord(c) > LAST_HANGUL: # if a character is a hangul
            result.append(c)
        else:            
            code = ord(c) - FIRST_HANGUL
            jongsung_index = code % N_JONGSUNGS
            code //= N_JONGSUNGS
            joongsung_index = code % N_JOONGSUNGS
            code //= N_JOONGSUNGS
            chosung_index = code

            result.append(CHOSUNGS[chosung_index])
            result.append(JOONGSUNGS[joongsung_index])
            result.append(JONGSUNGS[jongsung_index])
    
    return ''.join(result) 

def bunhae(s):
    return [to_jamo(i) for i in s]

In [ ]:
from tqdm import tqdm

def process_jamo(tokenized_corpus_fname, output_fname):
    toatal_lines = sum(1 for line in open(tokenized_corpus_fname, 'r', encoding='utf-8'))

    with open(tokenized_corpus_fname, 'r', encoding='utf-8') as f1, \
            open(output_fname, 'w', encoding='utf-8') as f2:

        for _, line in tqdm(enumerate(f1), total=toatal_lines):
            sentence = line.replace('\n', '').strip()
            processed_sentence = jamo_sentence(sentence)
            f2.writelines(processed_sentence + '\n')

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
from soynlp.hangle import compose, decompose, character_is_korean
doublespace_pattern = re.compile('\s+')


def jamo_sentence(sent):
    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '-' for c in cjj)
        return cjj_

    sent_ = []
    for char in sent:
        if character_is_korean(char):
            sent_.append(transform(char))
        else:
            sent_.append(char)
    sent_ = doublespace_pattern.sub(' ', ''.join(sent_))
    return sent_

def jamo_to_word(jamo):
    jamo_list, idx = [], 0
    while idx < len(jamo):
        if not character_is_korean(jamo[idx]):
            jamo_list.append(jamo[idx])
            idx += 1
        else:
            jamo_list.append(jamo[idx:idx + 3])
            idx += 3
    word = ""

    try:
      for jamo_char in jamo_list:
          if len(jamo_char) == 1:

              word += jamo_char

          elif jamo_char[2] == "-":
              
              try:
                word += compose(jamo_char[0], jamo_char[1], " ")
              except:
                word += jamo_char
                continue
          else:
              try:
                word += compose(jamo_char[0], jamo_char[1], jamo_char[2])
              except:
                word += jamo_char
                continue
    except:
      pass
      
    return word

In [ ]:
loaded_model1 = load_model('wordlstm/wordlstm.h5') # LSTM 형태소 모델 로드
loaded_model2 = load_model('jamolstm/jamolstm.h5') # LSTM 자모 모델 로드
loaded_model3 = load_model('jamocnn/jamocnn.h5') # CNN 자모 모델 로드
loaded_model4 = load_model('jamofastlstm/jamofastlstm.h5') # fasttext + lstm 자모 모델 로드
model_morphs = FastText.load("jamofastlstm/jamofastlstm.model") # fasttext + lstm 자모 모델 로드


with open('wordlstm/wordlstm.pkl', 'rb') as f: # LSTM 형태소 단어사전 로드
	tokenizer1 = pickle.load(f)
 
with open('jamolstm/jamolstm.pkl', 'rb') as f: # LSTM 자모 단어사전 로드
	tokenizer2 = pickle.load(f)
 
with open('jamocnn/jamocnn.pkl', 'rb') as f: # CNN 자모 단어사전 로드
	tokenizer3 = pickle.load(f)
 
 

In [ ]:
def predict_test(new_sentence):


  new_sentence1 = okt.morphs(new_sentence, stem=True) # 토큰화
  encoded1 = tokenizer1.texts_to_sequences([new_sentence1]) # 정수 인코딩
  pad_new1 = pad_sequences(encoded1, maxlen = 10) # 패딩
  score1 = float(loaded_model1.predict(pad_new1)) # 예측

  new_sentence2 = list("".join(bunhae(new_sentence)))
  encoded2 = tokenizer2.texts_to_sequences([new_sentence2]) # 정수 인코딩
  pad_new2 = pad_sequences(encoded2, maxlen = 100) # 패딩
  score2 = float(loaded_model2.predict(pad_new2)) # 예측

  new_sentence3 = list("".join(bunhae(new_sentence)))
  encoded3 = tokenizer3.texts_to_sequences([new_sentence3]) # 정수 인코딩
  pad_new3 = pad_sequences(encoded3, maxlen = 100) # 패딩
  score3 = float(loaded_model3.predict(pad_new3)) # 예측

  new_sentence4 = [new_sentence]
  pd.DataFrame(new_sentence4).to_csv('jamofastlstm/sentence.txt',index = False, header = False)
  sentence_txt = 'jamofastlstm/sentence.txt'
  new_sentence_txt = 'jamofastlstm/newsentence.txt'
  process_jamo(sentence_txt, new_sentence_txt)
  new_sentence_final = [sent.strip().split(" ") for sent in tqdm(open(new_sentence_txt, 'r', encoding='utf-8').readlines())]
  new_sentencelist = []
  for a in new_sentence_final:
    new_sentencelist2 = []
    for word in a:
      ft_modeled = model_morphs.wv.get_vector(word)
      new_sentencelist2.append(ft_modeled)
    new_sentencelist.append(new_sentencelist2)
  score4 = float(loaded_model4.predict(pad_sequences(new_sentencelist, maxlen = 15, padding='post', truncating = 'post', dtype = float )))

  score_average=(score1+score2+score3+score4)/4

  if(score_average > 0.5):
    print("{:.2f}% 확률로 욕설입니다.\n".format(score_average * 100))
  else:
    print("{:.2f}% 확률로 욕설이 아닙니다.\n".format((1 - score_average) * 100))


In [ ]:
predict_test('ㅈ같은련')

100%|██████████| 1/1 [00:00<00:00, 8594.89it/s]


94.17% 확률로 욕설입니다.

